# 幅優先探索

https://twitter.com/kdjp20/status/1364239574579302402

# 深さ優先探索

https://twitter.com/kdjp20/status/1364239574579302402

一点気をつけたいのが、depth。depth = 0を最初に宣言し、深さごとのループの最初に`depth += 1`するわけだが、一番深いレベルにおいては、新しくqに追加するnodeがないにも関わらず+1してしまっている。(自分の実装では、depthは、そのループで新しく探索される、つまり新しくqに入れられるnodeのdepthを表している。)

従って、一番深いdepth(0-index)を求めるには、ループの後に`depth -= 1`してやる必要がある。木のheightを求めるのであればこのdepthがそのまま答えになる。(`depth - 1 + 1 = depth`になるので)


# bisect

- bisect_left と bisect_rightの使い分け https://twitter.com/kdjp20/status/1424250217268871170
- bisect_left と bisect_rightの実装 https://twitter.com/kdjp20/status/1366717910815633410


# inorder-traversal in BST

### iteratively
https://twitter.com/kdjp20/status/1355171035402788867

In [ ]:
# recursively
def inorder_traveral(root):
    if root is None: return []
    return self.inorder_traveral(root.left) + [root.val] + self.inorder_traveral(root.right)

inorder(root)

# Reverse Linked List
→ https://www.youtube.com/watch?v=O0By4Zq0OFc

https://leetcode.com/problems/reverse-linked-list/

Given the head of a singly linked list, reverse the list, and return the reversed list.


#### Iterative solution
cursor的なものが走るイメージ

Time : O(N), Space: O(1)


In [ ]:
class Solution:
    def reverseList(self, head: ListNode) -> ListNode:
        prev = None
        curr = head
        
        while curr is not None:
            next = curr.next # keep it
            curr.next = prev
            
            prev = curr
            curr = next
        
        return prev

#### Recursive solution

引数のhead以降のlistをreverseし、新しいheadを返す。
recursion stackにより、spaceがO(N)かかってしまう。

Time : O(N), Space: O(N)

In [ ]:
class Solution:
    def reverseList(self, head: ListNode) -> ListNode:    
        # head is Noneとなるのは、もともとの呼び出し時のheadがNoneの場合のみ。
        # 基本的にlistにNodeが一つの場合にreturn headしている。
        # head.next is Noneの判定がないと、head.next.nextの判定(line10)でエラーを吐く。
        if head is None or head.next is None:
            return head
        
        new_head = self.reverseList(head.next)
        # head.nextはreverseされた後も指し示されるnodeは変わらず、reverseされたlistのtailのnodeを指している。
        head.next.next = head
        # headがこのfucntionで返すlistのtailのNodeになる
        head.next = None
        return new_head


## Segment Tree (top-downでrecursion)

https://leetcode.com/articles/a-recursive-approach-to-segment-trees-range-sum-queries-lazy-propagation/#  
https://leetcode.com/problems/range-sum-query-mutable/solution/ をもとに実装

In [230]:
import math
class SegmentTree:
    def __init__(self, nums):
        self.nums = nums
        self.l = [None] * len(nums) * 4    # enough slots(3だとダメなケースがあった......... https://leetcode.com/submissions/detail/509925769/)
        self._build_tree(0, 0, len(nums)-1)

    # Args:
    #   left_boundary, right_boundary: range of original "nums" which returned tree should be respoonsible for
    def _build_tree(self, curr_idx, curr_left, curr_right):

        if curr_left == curr_right:
            self.l[curr_idx] = self.nums[curr_left]
            return self.l[curr_idx]

        child_boundary = (curr_left + curr_right) // 2
        # tree(index = idx) has children: idx*2, idx*2+1
        left_sum = self._build_tree(curr_idx*2 + 1, curr_left, child_boundary)
        right_sum = self._build_tree(curr_idx*2 + 2, child_boundary+1, curr_right)
        self.l[curr_idx] = left_sum + right_sum
        return self.l[curr_idx]

    def update(self, idx, num):
        added_num = num - self.nums[idx]
        self.nums[idx] = num ######################## 忘れないように要注意

        curr_idx = 0
        curr_left = 0
        curr_right = len(self.nums)-1

        while curr_left < curr_right:
            self.l[curr_idx] += added_num

            # left child is responsible for range <= child_boundary
            # right child is responsible for child_buondary + 1 <= range
            child_boundary = (curr_left + curr_right) // 2
            if idx <= child_boundary:
                # go down to left
                curr_idx = curr_idx * 2 + 1
                curr_right = child_boundary
            else:
                # go down to right
                curr_idx = curr_idx * 2 + 2
                curr_left = child_boundary + 1

        # here, curr_right_boundary = curr_left_boundary = curr_idx
        self.l[curr_idx] += added_num

    def sumRange(self, left, right):
        curr_idx = 0
        return self._sum_range_inner(0, 0, len(self.nums)-1, left, right)

    # Args:
    #   curr_left, curr_right: range which current node is respoonsible for
    #   left, rightは最初に与えられた範囲(変えなくて良い)
    def _sum_range_inner(self, curr_idx, curr_left, curr_right, left, right): 

        if curr_right < left or right < curr_left:
            return 0
        if left <= curr_left and curr_right <= right:
            return self.l[curr_idx]

        # left child is responsible for range <= child_boundary
        # right child is responsible for child_buondary + 1 <= range
        child_boundary = (curr_left + curr_right) // 2

        left_child_sum = self._sum_range_inner(
            curr_idx * 2 + 1, curr_left, child_boundary, left, right)
        right_child_sum = self._sum_range_inner(
            curr_idx * 2 + 2, child_boundary+1, curr_right, left, right)
        return left_child_sum + right_child_sum


In [211]:
st.sumRange(1,2)

0 0 2 $ 1 2
1 0 1 $ 1 2
3 0 0 $ 1 2
4 1 1 $ 1 2
2 2 2 $ 1 2


5

# Topogical sort

トポロジカルソートは、グラフ理論において、有向非巡回グラフ（directed acyclic graph, DAG）の各ノードを順序付けして、どのノードもその出力辺の先のノードより前にくるように並べることである。有向非巡回グラフは必ずトポロジカルソートすることができる。

肝としては、indegreeに着目すること。

https://leetcode.com/problems/course-schedule-ii/

```
There are a total of n courses you have to take labelled from 0 to n - 1.
Some courses may have prerequisites, for example, if prerequisites[i] = [ai, bi] this means you must take the course bi before the course ai.
Given the total number of courses numCourses and a list of the prerequisite pairs, return the ordering of courses you should take to finish all courses.

If there are many valid answers, return any of them. If it is impossible to finish all courses, return an empty array.

[Example 1]
Input: numCourses = 2, prerequisites = [[1,0]]
Output: [0,1]
Explanation: There are a total of 2 courses to take. To take course 1 you should have finished course 0. So the correct course order is [0,1].
```

In [ ]:
from collections import defaultdict, deque

class Solution:
    # topological sort
    # Complexity Analysis
    #   Time :  O(v + e)
    #   Space:  O(v + e)
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:

        ### indegreesと、adj_listの用意
        indegrees = {}
        for i in range(numCourses):
            indegrees[i] = 0
        adj_list = defaultdict(set)
        for pre in prerequisites:
            adj_list[pre[1]].add(pre[0])
            indegrees[pre[0]] += 1
        
        ### qの中にindegree=0のverticesを収納する。
        ### ちなみに、queuじゃなくて、stackでも、setでも良い。(ただしどれを選ぶかで当然sort順は変わる)
        q = deque([])
        for course in indegrees:
            if indegrees[course] == 0:
                q.append(course)
        
        ###  coursesリストの中にsortされた要素を入れていく
        courses = []
        while len(q) > 0:
            out = q.popleft()
            courses.append(out)
            for course in adj_list[out]:
                indegrees[course] -= 1
                if indegrees[course] == 0:
                    q.append(course)
        if len(courses) < numCourses:   # 全てがcoursesの中に入っていない　= cycleが存在する
            return []
        return courses

## Minimum Spanning Tree

https://twitter.com/kdjp20/status/1423942609278406658

## DAGのCycle Detection

- DFS way https://twitter.com/kdjp20/status/1406198492515557379
- TopologicalSort way https://twitter.com/kdjp20/status/1406198628574580739

BFS wayは非推奨。理由は、See: https://stackoverflow.com/questions/2869647/why-dfs-and-not-bfs-for-finding-cycle-in-graphs

Ref:
- https://leetcode.com/problems/course-schedule
- https://leetcode.com/problems/all-paths-from-source-lead-to-destination/solution/

In [ ]:
class Solution:
    def minDifference(self, nums: List[int], queries: List[List[int]]) -> List[int]:
        max_seg = MaxSegmentTree(nums)
        min_seg = MinSegmentTree(nums)
        
        ret = []
        for min_b, max_b in queries:
            max_in_range = max_seg.maxRange(min_b, max_b)
            min_in_range = min_seg.minRange(min_b, max_b)
            ret.append(-1) if (max_in_range == min_in_range) else ret.append(max_in_range-min_in_range)
            
        return ret
        
        
import math
class MaxSegmentTree:
    def __init__(self, nums):
        self.nums = nums
        self.l = [None] * len(nums) * 4
        self._build_tree(0, 0, len(nums)-1)

    # Args:
    #   left_boundary, right_boundary: range of original "nums" which returned tree should be respoonsible for
    def _build_tree(self, curr_idx, curr_left, curr_right):

        if curr_left == curr_right:
            self.l[curr_idx] = self.nums[curr_left]
            return self.l[curr_idx]

        child_boundary = (curr_left + curr_right) // 2
        # tree(index = idx) has children: idx*2, idx*2+1
        left_max = self._build_tree(curr_idx*2 + 1, curr_left, child_boundary)
        right_max = self._build_tree(curr_idx*2 + 2, child_boundary+1, curr_right)
        self.l[curr_idx] = max(left_max, right_max)
        return self.l[curr_idx]

    def maxRange(self, left, right):
        curr_idx = 0
        return self._max_range_inner(0, 0, len(self.nums)-1, left, right)

    # Args:
    #   curr_left, curr_right: range which current node is respoonsible for
    #   left, rightは最初に与えられた範囲(変えなくて良い)
    def _max_range_inner(self, curr_idx, curr_left, curr_right, left, right): 

        if curr_right < left or right < curr_left:
            return 0
        if left <= curr_left and curr_right <= right:
            return self.l[curr_idx]

        # left child is responsible for range <= child_boundary
        # right child is responsible for child_buondary + 1 <= range
        child_boundary = (curr_left + curr_right) // 2

        left_child_max = self._max_range_inner(
            curr_idx * 2 + 1, curr_left, child_boundary, left, right)
        right_child_max = self._max_range_inner(
            curr_idx * 2 + 2, child_boundary+1, curr_right, left, right)
        return max(left_child_max, right_child_max)

    
class MinSegmentTree:
    def __init__(self, nums):
        self.nums = nums
        self.l = [None] * len(nums) * 4    # enough slots(3だとダメなケースがあった......... https://leetcode.com/submissions/detail/509925769/)
        self._build_tree(0, 0, len(nums)-1)

    # Args:
    #   left_boundary, right_boundary: range of original "nums" which returned tree should be respoonsible for
    def _build_tree(self, curr_idx, curr_left, curr_right):

        if curr_left == curr_right:
            self.l[curr_idx] = self.nums[curr_left]
            return self.l[curr_idx]

        child_boundary = (curr_left + curr_right) // 2
        # tree(index = idx) has children: idx*2, idx*2+1
        left_min = self._build_tree(curr_idx*2 + 1, curr_left, child_boundary)
        right_min = self._build_tree(curr_idx*2 + 2, child_boundary+1, curr_right)
        self.l[curr_idx] = min(left_min, right_min)
        return self.l[curr_idx]

    def minRange(self, left, right):
        curr_idx = 0
        return self._min_range_inner(0, 0, len(self.nums)-1, left, right)

    # Args:
    #   curr_left, curr_right: range which current node is respoonsible for
    #   left, rightは最初に与えられた範囲(変えなくて良い)
    def _min_range_inner(self, curr_idx, curr_left, curr_right, left, right): 

        if curr_right < left or right < curr_left:
            return 0
        if left <= curr_left and curr_right <= right:
            return self.l[curr_idx]

        # left child is responsible for range <= child_boundary
        # right child is responsible for child_buondary + 1 <= range
        child_boundary = (curr_left + curr_right) // 2
        
        left_child_min = self._min_range_inner(
            curr_idx * 2 + 1, curr_left, child_boundary, left, right)
        right_child_min = self._min_range_inner(
            curr_idx * 2 + 2, child_boundary+1, curr_right, left, right)
        return max(left_child_min, right_child_min)